Convert AlborEx positions data to JSON.

In [25]:
import alborex_functions
import alborexdata
import os
import geojson
import numpy as np
import netCDF4
import datetime

In [3]:
outputdir = "/home/ctroupin/Projects/Leaflet/AlborEx/data/"

# CTD

In [26]:
lonCTD1, latCTD1 = alborexdata.load_lonloat_ctdleg("../data/ctd_positions_Leg1.txt")
lonCTD2, latCTD2 = alborexdata.load_lonloat_ctdleg("../data/ctd_positions_Leg2.txt")

In [27]:
CTDlist = [[lat, lon] for lat, lon in zip(latCTD1, lonCTD1)]

In [28]:
with open(os.path.join(outputdir, 'CTD1.js'), 'w') as f:
    f.write("var CTD1list = " + str(CTDlist))

In [29]:
CTDlist = [[lat, lon] for lat, lon in zip(latCTD2, lonCTD2)]

In [30]:
with open(os.path.join(outputdir, 'CTD2.js'), 'w') as f:
    f.write("var CTD2list = " + str(CTDlist))

# Glider

In [75]:
gliderdir = "/home/ctroupin/DataOceano/Alborex/Glider/"
gliderfile1 = os.path.join(gliderdir, "dep0005_icoast00_ime-slcost000_L1_2014-05-25_data_dt.nc")
gliderfile2 = os.path.join(gliderdir, "dep0012_ideep00_ime-sldeep000_L1_2014-05-25_data_dt.nc")

In [76]:
lon1, lat1, depth1, time1 = alborex_functions.load_glider_coord(gliderfile1)
lon2, lat2, depth2, time2 = alborex_functions.load_glider_coord(gliderfile2)

In [70]:
def makeGeoJson(lon, lat, filename, varname, NN=100):
    """
    Create a geoJSON file containing the glider coordinates as a LineString object
    """

    # Remove masked values
    lon = np.ma.compressed(lon)
    lat = np.ma.compressed(lat)

    # Apply re-sampling (otherwise too many points)
    lonG, latG = lon[::NN], lat[::NN]

    # Create list of tuples
    Glider1list = [(lon, lat) for lon, lat in zip(lonG, latG)]

    # Create LineString object
    GliderGeoJson = geojson.LineString(Glider1list)

    # Write in new file
    with open(filename, 'w') as f:
        f.write("var {0} = ".format(varname))
        geojson.dump(GliderGeoJson, f)

In [77]:
makeGeoJson(lon1, lat1, os.path.join(outputdir, 'Glider1.js'), 'glider1', NN=100)

In [78]:
makeGeoJson(lon2, lat2, os.path.join(outputdir, 'Glider2.js'), 'glider2', NN=100)

# Drifters

We've created a list of deployments (text file) using [get_deployments_alborex](get_deployments_alborex.ipynb).

In [6]:
drifterfilelist = './deploymentList.txt'

Dates of interest for the extraction

In [56]:
dateinit = datetime.datetime(2014, 5, 25)
dateend = datetime.datetime(2014, 6, 15)
timeinit = (dateinit-datetime.datetime(1970,1,1)).total_seconds()
timeend = (dateend-datetime.datetime(1970,1,1)).total_seconds()

In [66]:
lonall, latall = [], []
with open(drifterfilelist, 'r') as f:
    for drifterfile in f:
        with netCDF4.Dataset(drifterfile) as nc:
            time = nc.variables['time'][:]
            timeunits = nc.variables['time'].units
            goodtime = np.where(np.logical_and((time > timeinit),(time <= timeend)))[0]
            
            # Only take values in the considered period
            lon = nc.variables['LON'][goodtime]
            lat = nc.variables['LAT'][goodtime]
            QClon = nc.variables['QC_LON'][goodtime]
            QClat = nc.variables['QC_LAT'][goodtime]
            # Only take good coordinates
            goodcoords = np.where(np.logical_and((QClon==1),(QClat==1)))[0]
            lon = lon[goodcoords]
            lat = lat[goodcoords]
            
            lonall.append(lon.tolist())
            latall.append(lat.tolist())
            #print(lon.min(), lon.max(), lat.min(), lat.max())

In [67]:
# Create list of tuples
DrifterList = [[(lon, lat) for lon, lat in zip(londep, latdep)] for londep, latdep in zip(lonall, latall)]

In [68]:
DrifterGeojson = geojson.MultiLineString(DrifterList)  

In [69]:
with open(os.path.join(outputdir, 'drifters.js'), 'w') as f:
    f.write("var drifters = ")
    geojson.dump(DrifterGeojson, f)

# Profilers

Same as drifters.

In [77]:
profilerfilelist = './alborexProfilerList.txt'

In [99]:
lonall, latall = [], []
with open(profilerfilelist, 'r') as f:
    for profilerfile in f:
        with netCDF4.Dataset(profilerfile) as nc:
            time = nc.variables['time'][:]
            timeunits = nc.variables['time'].units
            goodtime = np.where(np.logical_and((time > timeinit),(time <= timeend)))[0]
            
            # Only take values in the considered period
            lon = nc.variables['LON'][goodtime[1:]]
            lat = nc.variables['LAT'][goodtime[1:]]
            """
            QClon = nc.variables['QC_LON'][goodtime]
            QClat = nc.variables['QC_LAT'][goodtime]
            # Only take good coordinates
            goodcoords = np.where(np.logical_and((QClon==1),(QClat==1)))[0]
            lon = lon[goodcoords]
            lat = lat[goodcoords]
            """
            lonall.append(lon.tolist())
            latall.append(lat.tolist())
            #print(lon.min(), lon.max(), lat.min(), lat.max())

In [100]:
# Create list of tuples
ProfilerList = [[(lon, lat) for lon, lat in zip(londep, latdep)] for londep, latdep in zip(lonall, latall)]
ProfilerGeojson = geojson.MultiLineString(ProfilerList)  
with open(os.path.join(outputdir, 'profilers.js'), 'w') as f:
    f.write("var profilers = ")
    geojson.dump(ProfilerGeojson, f)

In [80]:
with netCDF4.Dataset(profilerfile.rstrip()) as nc:
    print(nc.variables)

OrderedDict([('trajectory', <class 'netCDF4._netCDF4.Variable'>
|S1 trajectory(maxStrlen64)
    cf_role: trajectory_id
unlimited dimensions: 
current shape = (64,)
filling off
), ('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    units: seconds since 1970-01-01 00:00:00
    _FillValue: nan
    axis: T
    calendar: gregorian
unlimited dimensions: time
current shape = (71,)
filling off
), ('LAT', <class 'netCDF4._netCDF4.Variable'>
float64 LAT(time)
    standard_name: latitude
    long_name: Latitude
    units: degrees_north
    _FillValue: nan
    axis: Y
    valid_min: -90.0
    valid_max: 90.0
    reference_datum: geographical coordinates, WGS84 projection
unlimited dimensions: time
current shape = (71,)
filling off
), ('LON', <class 'netCDF4._netCDF4.Variable'>
float64 LON(time)
    standard_name: longitude
    long_name: Longitude
    units: degrees_east
    _FillValue: nan
    axis: X
    valid_min: -180.0
    valid_max: 180.0
    reference